In [1]:
from transformers.models.vit.modeling_vit import ViTModel, ViTLayer, ViTEncoder, ViTConfig
from torch import nn
import torch
from typing import Optional
from transformers import ViTForImageClassification
import torch.nn.functional as F
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
from torch.utils.data import Dataset, DataLoader, Subset
from transformers import AutoImageProcessor, AutoModelForImageClassification
import torchvision
from tqdm import tqdm
import os
import torch.optim as optim
from ptflops import get_model_complexity_info
import pandas as pd

/user/ai21btech11011/miniforge3/envs/himanshu_mp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Custom dataset class for CIFAR-100 that integrates with AutoImageProcessor for processing
class CIFAR100Dataset(Dataset):
    def __init__(self, root, train=True, size=None, processor=None):
        self.dataset = torchvision.datasets.CIFAR100(root=root, train=train, download=True)           # Load CIFAR-100 dataset from torchvision
        if size is not None:
            self.dataset = Subset(self.dataset, torch.arange(size))                                  # Optionally, load only count number of samples
        
        self.processor = processor                                                                    # Store the processor (used to preprocess images before passing to the model)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image, label = self.dataset[idx] 
        
        # Preprocess the image and convert it to tensor format suitable for the model
        inputs = self.processor(images=image, return_tensors="pt")
        return inputs['pixel_values'].squeeze(0), label  
    

class ModifiedViTLayer(ViTLayer):
    def __init__(self, config):
        super().__init__(config)
        self.hidden_size = config.hidden_size
        layer_sizes = [self.hidden_size, 64, 1]

        mlp_layers = []
        for i in range(len(layer_sizes) - 1):
            if i < len(mlp_layers) - 2:
                mlp_layers.extend([nn.Linear(layer_sizes[i], layer_sizes[i + 1]), nn.ReLU()])
            else:
                mlp_layers.extend([nn.Linear(layer_sizes[i], layer_sizes[i + 1]), nn.Sigmoid()])                

        self.mlp_layer = nn.Sequential(*mlp_layers)
        self.threshold = 0.05
        self.loss = 0
        self.skip_ratio = 0


    def forward(self, hidden_states: torch.Tensor,
        head_mask: Optional[torch.Tensor] = None,
        output_attentions: bool = False, compute_cosine = True):

        mlp_output = self.mlp_layer(hidden_states[:, 1:])
        boolean_mask = (mlp_output >= self.threshold).squeeze(-1)
        cls_col = torch.ones((hidden_states.shape[0], 1), dtype=torch.bool).to(boolean_mask.device)
        boolean_mask = torch.cat((cls_col, boolean_mask), dim=1)

        output = hidden_states.clone()
        batch_size = hidden_states.shape[0]
        for i in range(batch_size):
            output[i][boolean_mask[i]] = super().forward(hidden_states[i][boolean_mask[i]].unsqueeze(0))[0].squeeze(0)

        if self.training or compute_cosine:
            real_output = super().forward(hidden_states[:, 1:])[0]
            cos_similarity = (F.cosine_similarity(real_output, hidden_states[:, 1:], dim=-1) + 1) / 2
            
            self.loss = nn.MSELoss()(cos_similarity, 1 - mlp_output.squeeze(-1))
            self.mlp_accuracy_arr = ((1 - cos_similarity - self.threshold) * (mlp_output.squeeze(-1) - self.threshold) > 0)
        else:
            self.loss = 0
            
        self.skip_ratio = torch.sum(~boolean_mask)
        return (output, )


class ModifiedViTEncoder(ViTEncoder):
    def __init__(self, config: ViTConfig):
        super().__init__(config)
        self.layer = nn.ModuleList([ModifiedViTLayer(config) for _ in range(config.num_hidden_layers)])


class ModifiedViTModel(ViTModel):
    def __init__(self, config: ViTConfig):
        super().__init__(config)
        self.encoder = ModifiedViTEncoder(config)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, pixel_values, output_attentions=False, output_hidden_states=False, return_dict=True):
        outputs = super().forward(pixel_values, output_attentions=output_attentions, output_hidden_states=output_hidden_states, return_dict=return_dict)
        logits = self.classifier(outputs['last_hidden_state'][:, 0])
        output = lambda: None
        setattr(output, 'logits', logits)

        return output

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Calculating Flops
input_size = (3, 224, 224)
pretrained_cifar_model = AutoModelForImageClassification.from_pretrained("Ahmed9275/Vit-Cifar100")
flops_original, params = get_model_complexity_info(pretrained_cifar_model, input_size, as_strings=True, print_per_layer_stat=False)
print(flops_original)

cuda
16.87 GMac


In [12]:
# Function to evaluate the model on the test data
def test(model, dataloader, full_testing=False, progress=True):
    model.eval()
    total_correct = 0
    each_layer_skip = torch.zeros(len(model.encoder.layer))
    if full_testing:
        total_correct_mlp = torch.zeros(len(model.encoder.layer))
        total_mlp_count = torch.zeros(len(model.encoder.layer))
    data_size = len(dataloader.dataset)
    with torch.no_grad():             # Disable gradient computation for efficiency
        if progress:
            dataloader = tqdm(dataloader, desc=f"Testing", leave=False)
        for (inputs, labels) in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            if full_testing:
                outputs = model(inputs)
            else:
                outputs = model(inputs)
            logits = outputs.logits

            predicted_class_indices = logits.argmax(dim=-1)    # Get the class index with the highest logit score

            # Track the skip ratios from each layer
            for i, vit_layer in enumerate(model.encoder.layer):
                each_layer_skip[i] += vit_layer.skip_ratio.item()
                if full_testing:
                    total_correct_mlp[i] += (vit_layer.mlp_accuracy_arr).sum().item()
                    total_mlp_count[i] += vit_layer.mlp_accuracy_arr.numel()
            total_correct += torch.sum(predicted_class_indices == labels).item()

    if full_testing:
        print(f"correct prediction ratio: {total_correct_mlp / total_mlp_count}")
    print(f"Skip ratio: {each_layer_skip / data_size}")
    return total_correct / data_size



# Training function to train the model over multiple epochs
def train(model, train_loader, test_loader, num_epochs=10, loss_type='both', lr=1e-4, progress=True):
    
    results = {"epoch": [], "train_loss": [], "test_accuracy": []}
    
    model.train()
    criterion = nn.CrossEntropyLoss()
    cosine_loss_ratio = 1.5

    optimizer = optim.Adam([param for param in model.parameters() if param.requires_grad], lr=lr)
    temp_train_loader = train_loader
    for epoch in range(num_epochs):
        model.train()  # Re-enable training mode at the start of each epoch
        running_loss = 0.0
        if progress:
            train_loader = tqdm(train_loader, desc=f"Epoch [{epoch + 1}/{num_epochs}]", leave=False)

        for batch_idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)                

            # Forward pass to get the classification logits, loss calculation
            logits = model(inputs).logits

            if loss_type in ['classification', 'both']:
                classification_loss = criterion(logits, labels)

            # Accumulate cosine similarity losses from each DHSLayer
            if loss_type in ['cosine', 'both']:
                cosine_loss = 0.0
                for i, vit_layer in enumerate(model.encoder.layer):
                    cosine_loss += vit_layer.loss
            
            # Calculate the total loss as a weighted sum of classification and cosine similarity losses
            if loss_type == 'classification':
                total_loss = classification_loss
            elif loss_type == 'cosine':
                total_loss = cosine_loss
            elif loss_type == 'both':
                total_loss = classification_loss + cosine_loss_ratio * cosine_loss
            
            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()

            # Accumulate running loss for this batch
            running_loss += total_loss.item()

            # Update the progress bar with current batch loss
            if progress:
                if (batch_idx + 1) % 5 == 0:
                    train_loader.update(5)
                train_loader.set_postfix({'batch_loss': total_loss.item()})


        train_loss = running_loss / len(train_loader)
        test_accuracy = test(model, test_loader, full_testing=True, progress=progress)
        
        results["epoch"].append(epoch + 1)
        results["train_loss"].append(train_loss)
        results["test_accuracy"].append(test_accuracy)
        
        # Print or log results for the current epoch
        print(f"Epoch {epoch + 1}/{num_epochs}, Avg. Loss: {train_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

    return results

In [13]:
# Define the training parameters
load_pretrained = False            # Flag to load a pre-trained model
data_path = 'data'
train_size = 10000
test_size = 1000
pretrained_model_path = os.path.join('models', 'mlp_class_10000.pth')



config = ViTConfig.from_pretrained("Ahmed9275/Vit-Cifar100")
config.num_labels = len(config.id2label)
modified_vit_model = ModifiedViTModel(config).to(device)
cifar_processor = AutoImageProcessor.from_pretrained("Ahmed9275/Vit-Cifar100")   # Initialize the image processor for CIFAR-100 data pre-processing


train_dataset = CIFAR100Dataset(root=data_path, train=True, size=train_size, processor=cifar_processor)
test_dataset = CIFAR100Dataset(root=data_path, train=False, size=test_size, processor=cifar_processor)

# Define the function to run training with given hyperparameters
def run_experiment(batch_size, lr, num_epochs, loss_type):
    # Print the hyperparameters
    print(f"\n ---------- Running experiment with batch_size={batch_size}, lr={lr}, num_epochs={num_epochs}, loss_type={loss_type} ---------- \n")


    # -----------   Copying Weights    ------------  #
    if not load_pretrained:
        new_state_dict = {}
        for key in pretrained_cifar_model.state_dict().keys():
            new_key = key.replace('vit.', '')
            new_state_dict[new_key] = pretrained_cifar_model.state_dict()[key]

        modified_vit_model.load_state_dict(new_state_dict, strict=False)
    else:
        modified_vit_model.load_state_dict(torch.load(pretrained_model_path))

    modified_vit_model.to(device)
    

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    # Making grad true or false depending on training type
    for param in modified_vit_model.parameters():
        param.requires_grad = False

    if loss_type in ['cosine', 'both']:
        for layer in modified_vit_model.encoder.layer:
            for param in layer.mlp_layer.parameters():
                param.requires_grad = True

    if loss_type in ['classification', 'both']:
        for param in modified_vit_model.classifier.parameters():
            param.requires_grad = True


    # Training the model
    results = train(modified_vit_model, train_loader, test_loader, num_epochs=num_epochs, loss_type=loss_type, lr=lr, progress=True)
    
    input_size = (3, 224, 224)
    flops, params = get_model_complexity_info(modified_vit_model, input_size, as_strings=True, print_per_layer_stat=False)
    
    print(f"FLOPS for this modified model: {flops}")
    # Extracting only the numerical part and converting to float
    flops = float(flops.split()[0].replace(',', ''))
    flops_original, params = get_model_complexity_info(pretrained_cifar_model, input_size, as_strings=True, print_per_layer_stat=False)
    flops_original = float(flops_original.split()[0].replace(',', ''))
    print(f"Ratio of FLOPS: {flops / flops_original} \n")


    return results

Files already downloaded and verified
Files already downloaded and verified


In [14]:
# List of hyperparameters combinations to try
experiments = [
    {"batch_size": 64, "lr": 1e-4, "num_epochs": 15, "loss_type": "cosine"},
    {"batch_size": 64, "lr": 2e-4, "num_epochs": 15, "loss_type": "both"},
    # Add more combinations here...
]


# Run all experiments and store the results in a DataFrame
results_df = pd.DataFrame()
for exp in experiments:
    exp_results = run_experiment(**exp)
    exp_df = pd.DataFrame(exp_results)
    exp_df["batch_size"] = exp["batch_size"]
    exp_df["lr"] = exp["lr"]
    exp_df["num_epochs"] = exp["num_epochs"]
    exp_df["loss_type"] = exp["loss_type"]
    results_df = pd.concat([results_df, exp_df], ignore_index=True)


 ---------- Running experiment with batch_size=64, lr=0.0001, num_epochs=15, loss_type=cosine ---------- 



correct prediction ratio: tensor([1.0000, 1.0000, 0.9998, 0.9059, 0.8190, 0.5589, 0.2814, 0.1568, 0.2886,
        0.3599, 0.8652, 0.8856])
Skip ratio: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
Epoch 1/15, Avg. Loss: 0.6363, Test Accuracy: 0.8990


correct prediction ratio: tensor([1.0000, 1.0000, 0.9998, 0.9059, 0.8190, 0.5589, 0.2814, 0.1568, 0.2886,
        0.3599, 0.8652, 0.8856])
Skip ratio: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
Epoch 2/15, Avg. Loss: 0.0892, Test Accuracy: 0.8990


correct prediction ratio: tensor([1.0000, 1.0000, 0.9998, 0.9059, 0.8190, 0.5589, 0.2814, 0.1568, 0.2886,
        0.3599, 0.8652, 0.8856])
Skip ratio: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
Epoch 3/15, Avg. Loss: 0.0365, Test Accuracy: 0.8990


correct prediction ratio: tensor([1.0000, 1.0000, 0.9998, 0.9059, 0.8190, 0.5589, 0.2814, 0.1568, 0.2886,
        0.3661, 0.8666, 0.8856])
Skip ratio: tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        1.2690, 0.0000, 0.0000])
Epoch 4/15, Avg. Loss: 0.0215, Test Accuracy: 0.8990


...

correct prediction ratio: tensor([1.0000, 1.0000, 0.9998, 0.9123, 0.8551, 0.7730, 0.7916, 0.8593, 0.8295,
        0.7982, 0.9872, 0.9923])
Skip ratio: tensor([  0.0000,   0.0000,   0.0000,   5.2270,  13.6580,  60.0800, 158.1110,
        177.7230, 147.2300, 140.2590,   6.4380,   3.8690])
Epoch 11/15, Avg. Loss: 0.4927, Test Accuracy: 0.7580


correct prediction ratio: tensor([1.0000, 1.0000, 0.9998, 0.9125, 0.8559, 0.7757, 0.7950, 0.8563, 0.8311,
        0.7929, 0.9859, 0.9931])
Skip ratio: tensor([  0.0000,   0.0000,   0.0000,   5.5590,  17.6310,  54.2530, 163.5100,
        177.0350, 149.6110, 136.3080,   6.3070,   4.1620])
Epoch 12/15, Avg. Loss: 0.4692, Test Accuracy: 0.7600


correct prediction ratio: tensor([1.0000, 1.0000, 0.9998, 0.9126, 0.8567, 0.7794, 0.7985, 0.8546, 0.8348,
        0.7973, 0.9854, 0.9927])
Skip ratio: tensor([  0.0000,   0.0000,   0.0000,   5.9430,  15.9650,  60.6470, 163.0590,
        173.5890, 151.0300, 138.9710,   6.4560,   4.2980])
Epoch 13/15, Avg. Loss: 0.4502, Test Accuracy: 0.7530


correct prediction ratio: tensor([1.0000, 1.0000, 0.9998, 0.9127, 0.8565, 0.7770, 0.8027, 0.8591, 0.8365,
        0.7970, 0.9866, 0.9928])
Skip ratio: tensor([  0.0000,   0.0000,   0.0000,   5.2330,  15.4600,  56.3980, 163.2920,
        176.6880, 155.0580, 142.7140,   6.6720,   4.8190])
Epoch 14/15, Avg. Loss: 0.4394, Test Accuracy: 0.7500


correct prediction ratio: tensor([1.0000, 1.0000, 0.9998, 0.9129, 0.8569, 0.7794, 0.8009, 0.8566, 0.8384,
        0.7975, 0.9855, 0.9924])
Skip ratio: tensor([  0.0000,   0.0000,   0.0000,   5.4960,  16.2470,  57.6170, 163.0390,
        173.1390, 154.7400, 135.6300,   6.7210,   4.4120])
Epoch 15/15, Avg. Loss: 0.4281, Test Accuracy: 0.7460
FLOPS for this modified model: 31.63 GMac
Ratio of FLOPS: 1.874925903971547 



In [11]:
results_df 

,epoch,train_loss,test_accuracy,batch_size,lr,num_epochs,loss_type
0,1,0.747893,0.899,64,0.0001,15,cosine
